In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup as bs
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm
import concurrent
import json
import urllib.parse
import warnings
warnings.filterwarnings("ignore")

In [ ]:
urls = ['https://soroush.app/sitemap-1.xml',
        'https://soroush.app/sitemap-2.xml',
        'https://soroush.app/sitemap-3.xml',
        'https://soroush.app/sitemap-4.xml',
        'https://soroush.app/sitemap-5.xml',
        'https://soroush.app/sitemap-6.xml',
        'https://soroush.app/sitemap-7.xml',
        'https://soroush.app/sitemap-8.xml']

links = pd.DataFrame()

In [ ]:
for i,s in enumerate(urls):
    xml = pd.read_xml(s)
    print(len(xml))
    links = links.append(xml)
    print(len(links))
    print(i)

In [ ]:
links.reset_index(inplace=True)
links.drop(["index"], axis=1, inplace=True)
links["url"]=links["loc"].apply(urllib.parse.unquote)
links.drop(['changefreq', 'lastmod'], axis=1, inplace=True)
links.drop_duplicates(inplace=True)

In [ ]:
links.to_csv('urls2.csv', encoding='utf-8', index=False)

In [2]:
sure = pd.read_csv("urls2.csv")
sure = sure['url']
sure

0                                      https://soroush.app/
1                              https://soroush.app/چلو-کباب
2                               https://soroush.app/پیش-غذا
3                            https://soroush.app/نان-و-خمیر
4                          https://soroush.app/روغن-زیتون-4
                                ...                        
224565            https://soroush.app/روغن-حیوانی-400م-اصیل
224566    https://soroush.app/بیسکوئیت-جو-رژیمی-استوانه-...
224567    https://soroush.app/شامپو-موهای-آسیب-دیده-حاوی...
224568    https://soroush.app/باطری-شارژی-قلم-2-عددی-260...
224569    https://soroush.app/سس-مایونز-سالادی-تند-400گ-هنز
Name: url, Length: 224570, dtype: object

In [3]:
# Define a function to get status codes for each URL using requests library
def get_futures(url):
    try:
        response = requests.get(url)
        soup = bs(response.content, 'lxml')
        name = soup.find('div', {'class':'product-name'})
        if name:
            name = name.text
            img = soup.find('a',{'id':'zoom1'})['data-full-image-url']
            breadcrumb = soup.find('div',{'class':'breadcrumb'}).ul.text
            brand = soup.find('div',{'class':'manufacturers'})
            brand= brand.find('span',{'class':'value'}).text if brand else ' '
            id = soup.find('div',{'class':'sku'}).find('span',{'class':'value'}).text.split('-')[0]
            return name, img, breadcrumb, brand, id
        else:
            return None, None, None, None, None

    except Exception as e:
        return str(e)

# Define a callback function to update the progress bar
def progress_callback(future):
    pbar.update(1)

# Set up a progress bar
pbar = tqdm(total=len(sure))


# Use ThreadPoolExecutor and callback function to parallelize the apply function and update progress bar
with ThreadPoolExecutor(max_workers=10) as executor:
    futures = []
    for url in sure:
        futures.append(executor.submit(get_futures, url))

    for future in concurrent.futures.as_completed(futures):
        status_code = future.result()
        progress_callback(future)

# Append the status codes list as a new column in the original dataframe
sure['نام محصول'] = [future.result()[0] for future in futures]
sure['img'] = [future.result()[1] for future in futures]
sure['دسته بندی'] = [future.result()[2] for future in futures]
sure['برند'] = [future.result()[3] for future in futures]
sure['شناسه'] = [future.result()[4] for future in futures]
sure['شناسه'] = [future.result()[4] for future in futures]

# Close the progress bar
pbar.close()

# Print the updated dataframe
print(sure)

100%|██████████| 224570/224570 [18:54:18<00:00,  3.30it/s]    


0                                         https://soroush.app/
1                                 https://soroush.app/چلو-کباب
2                                  https://soroush.app/پیش-غذا
3                               https://soroush.app/نان-و-خمیر
4                             https://soroush.app/روغن-زیتون-4
                                   ...                        
نام محصول    [None, None, None, None, None, None, None, Non...
img          [None, None, None, None, None, None, None, Non...
دسته بندی    [None, None, None, None, None, None, None, Non...
برند         [None, None, None, None, None, None, None, Non...
شناسه        [None, None, None, None, None, None, None, Non...
Name: url, Length: 224575, dtype: object


In [4]:
sure.to_csv('sure.csv', encoding='utf-8')

In [5]:
sure

0                                         https://soroush.app/
1                                 https://soroush.app/چلو-کباب
2                                  https://soroush.app/پیش-غذا
3                               https://soroush.app/نان-و-خمیر
4                             https://soroush.app/روغن-زیتون-4
                                   ...                        
نام محصول    [None, None, None, None, None, None, None, Non...
img          [None, None, None, None, None, None, None, Non...
دسته بندی    [None, None, None, None, None, None, None, Non...
برند         [None, None, None, None, None, None, None, Non...
شناسه        [None, None, None, None, None, None, None, Non...
Name: url, Length: 224575, dtype: object

In [20]:
sure = pd.read_csv("urls2.csv")
sure = sure['url']
sure

0                                      https://soroush.app/
1                              https://soroush.app/چلو-کباب
2                               https://soroush.app/پیش-غذا
3                            https://soroush.app/نان-و-خمیر
4                          https://soroush.app/روغن-زیتون-4
                                ...                        
224565            https://soroush.app/روغن-حیوانی-400م-اصیل
224566    https://soroush.app/بیسکوئیت-جو-رژیمی-استوانه-...
224567    https://soroush.app/شامپو-موهای-آسیب-دیده-حاوی...
224568    https://soroush.app/باطری-شارژی-قلم-2-عددی-260...
224569    https://soroush.app/سس-مایونز-سالادی-تند-400گ-هنز
Name: url, Length: 224570, dtype: object

In [23]:
sure = pd.DataFrame(sure)

In [24]:
sure

,url
0,https://soroush.app/
1,https://soroush.app/چلو-کباب
2,https://soroush.app/پیش-غذا
3,https://soroush.app/نان-و-خمیر
4,https://soroush.app/روغن-زیتون-4
...,...
224565,https://soroush.app/روغن-حیوانی-400م-اصیل
224566,https://soroush.app/بیسکوئیت-جو-رژیمی-استوانه-...
224567,https://soroush.app/شامپو-موهای-آسیب-دیده-حاوی...
224568,https://soroush.app/باطری-شارژی-قلم-2-عددی-260...


In [25]:
sure['نام محصول'] = [future.result()[0] for future in futures]
sure['img'] = [future.result()[1] for future in futures]
sure['دسته بندی'] = [future.result()[2] for future in futures]
sure['برند'] = [future.result()[3] for future in futures]
sure['شناسه'] = [future.result()[4] for future in futures]
sure['تصویر'] = [future.result()[4] for future in futures]

In [26]:
sure

,url,نام محصول,img,دسته بندی,برند,شناسه,تصویر
0,https://soroush.app/,None,None,None,None,None,None
1,https://soroush.app/چلو-کباب,None,None,None,None,None,None
2,https://soroush.app/پیش-غذا,None,None,None,None,None,None
3,https://soroush.app/نان-و-خمیر,None,None,None,None,None,None
4,https://soroush.app/روغن-زیتون-4,None,None,None,None,None,None
...,...,...,...,...,...,...,...
224565,https://soroush.app/روغن-حیوانی-400م-اصیل,روغن حیوانی 400میل اصیل,https://soroush.app/images/thumbs/000/0009944_...,خانه /غذا و نوشیدنی /خواربار و نان /روغن /روغن...,,6291113102444,6291113102444
224566,https://soroush.app/بیسکوئیت-جو-رژیمی-استوانه-...,بیسکوئیت جو رژیمی استوانه ای 280گ گالونبرند: گ...,https://soroush.app/images/thumbs/004/0043980_...,خانه /غذا و نوشیدنی /تنقلات /بیسکویت و ویفر /ب...,گالون | Gullon,8410376043686,8410376043686
224567,https://soroush.app/شامپو-موهای-آسیب-دیده-حاوی...,شامپو موهای آسیب دیده حاوی روغن زیتون 400م گارنیر,https://soroush.app/images/default-image.png,خانه /شامپو موهای آسیب دیده حاوی روغن زیتون 40...,,3610340027130,3610340027130
224568,https://soroush.app/باطری-شارژی-قلم-2-عددی-260...,باطری شارژی قلم 2 عددی 2600 میلی آمپر وارتابرن...,https://soroush.app/images/thumbs/001/0014427_...,خانه /خانه و زندگی /ابزار و سایر لوازم /لوازم ...,وارتا|varta,4008496745937,4008496745937


In [28]:
sure.to_csv("sure2.csv", encoding='utf-8')

In [29]:
sure.to_excel("sure.xlsx", encoding='utf-8')

In [45]:
mask =(sure['img'] == None) & (sure['شناسه'] == None) & (sure['برند'] == None) & (sure['نام محصول'] == None)

In [86]:
2+2

4

In [87]:
pert = sure[sure['برند'].isna()==True]

pert.reset_index(drop=True, inplace=True)

pert.drop('img', axis=1, inplace=True)
pert




,url,نام محصول,دسته بندی,برند,شناسه,تصویر
0,https://soroush.app/,None,None,None,None,None
1,https://soroush.app/چلو-کباب,None,None,None,None,None
2,https://soroush.app/پیش-غذا,None,None,None,None,None
3,https://soroush.app/نان-و-خمیر,None,None,None,None,None
4,https://soroush.app/روغن-زیتون-4,None,None,None,None,None
...,...,...,...,...,...,...
216326,https://soroush.app/هورام,None,None,None,None,None
216327,https://soroush.app/هومینا-hoomina,None,None,None,None,None
216328,https://soroush.app/هیمالیا-himalaya,None,None,None,None,None
216329,https://soroush.app/یک-ویک-11,None,None,None,None,None


In [94]:
pert.loc[216330]

url          https://soroush.app/یونسی
نام محصول                         None
دسته بندی                         None
برند                              None
شناسه                             None
تصویر                             None
Name: 216330, dtype: object

In [91]:
pert.to_excel('notAproduct.xlsx', encoding='utf-8')

In [2]:
ax = pd.read_csv('sure.csv')

In [103]:
failed = ax[ax['شناسه'].isna()]

In [108]:
failed.drop(['Unnamed: 0'], axis=1 ,inplace=True)

In [110]:
failed.reset_index(drop=True, inplace=True)

In [116]:
failed.to_csv('failed.csv', encoding='utf-8')

In [117]:
failed.to_excel('failed.xlsx', encoding='utf-8')

In [3]:
product = ax[ax['شناسه'].isna()== False]
product

,Unnamed: 0,url,نام محصول,img,دسته بندی,برند,شناسه,تصویر
321,321,https://soroush.app/صابون-رزماری-و-آویشن-پالمو...,صابون رزماری و آویشن پالمولیو مدل Herbal Extra...,https://soroush.app/images/thumbs/002/0024315_...,خانه /سلامت و زیبایی /لوازم بهداشتی /بهداشت و ...,پالمولیو | Palmolive,6281001342076,6281001342076
322,322,https://soroush.app/نرم-کننده-گلرنگ-پروتئینه-ص...,نرم کننده پروتئینه گلرنگ رنگ صورتی وزن 300 گرم...,https://soroush.app/images/thumbs/002/0024378_...,خانه /سلامت و زیبایی /لوازم بهداشتی /شامپو و م...,گلرنگ | Golrang,6260010519204,6260010519204
323,323,https://soroush.app/مایع-سفیدکننده-معمولی-جدید...,مایع سفیدکننده معمولی جدید اکتیو حجم 1 لیتربرن...,https://soroush.app/images/thumbs/002/0024478_...,خانه /سلامت و زیبایی /مواد شوینده /شوینده سطوح...,اکتیو | Active,6262825800195,6262825800195
324,324,https://soroush.app/خمیر-دندان-مریدنت-سفید-کنن...,خمیر دندان مریدنت سفید کننده 100میل کاملبرند: ...,https://soroush.app/images/thumbs/000/0000981_...,خانه /سلامت و زیبایی /لوازم بهداشتی /بهداشت ده...,مریدنت | Merident,6260010549300,6260010549300
325,325,https://soroush.app/مایع-لوله-بازکن-اتک1-لیتری,مایع لوله بازکن اتک1 لیتریبرند: اَتک | Attek,https://soroush.app/images/thumbs/000/0001007_...,خانه /سلامت و زیبایی /مواد شوینده /مایع لوله ب...,اَتک | Attek,6260012941836,6260012941836
...,...,...,...,...,...,...,...,...
224565,224565,https://soroush.app/روغن-حیوانی-400م-اصیل,روغن حیوانی 400میل اصیل,https://soroush.app/images/thumbs/000/0009944_...,خانه /غذا و نوشیدنی /خواربار و نان /روغن /روغن...,,6291113102444,6291113102444
224566,224566,https://soroush.app/بیسکوئیت-جو-رژیمی-استوانه-...,بیسکوئیت جو رژیمی استوانه ای 280گ گالونبرند: گ...,https://soroush.app/images/thumbs/004/0043980_...,خانه /غذا و نوشیدنی /تنقلات /بیسکویت و ویفر /ب...,گالون | Gullon,8410376043686,8410376043686
224567,224567,https://soroush.app/شامپو-موهای-آسیب-دیده-حاوی...,شامپو موهای آسیب دیده حاوی روغن زیتون 400م گارنیر,https://soroush.app/images/default-image.png,خانه /شامپو موهای آسیب دیده حاوی روغن زیتون 40...,,3610340027130,3610340027130
224568,224568,https://soroush.app/باطری-شارژی-قلم-2-عددی-260...,باطری شارژی قلم 2 عددی 2600 میلی آمپر وارتابرن...,https://soroush.app/images/thumbs/001/0014427_...,خانه /خانه و زندگی /ابزار و سایر لوازم /لوازم ...,وارتا|varta,4008496745937,4008496745937


In [121]:
product.drop('Unnamed: 0',axis=1, inplace=True)

In [122]:
product.reset_index(inplace=True, drop=True)

In [124]:
product.to_csv('product.csv', encoding='utf-8')

In [125]:
product.to_excel('product.xlsx', encoding='utf-8', index=False)

In [5]:
import requests
import os
import concurrent.futures

def get_image_from_url(url, name):
    try:
        response = requests.get(url)

        with open(f'./img/{name}.jpg', 'wb') as file:
            file.write(response.content)
    except Exception as e:
        print(e,"+++++", url)



def download_all_images(df):
    urls = df['img'].tolist()
    names = df['شناسه'].tolist()
    futures = []
    with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
        for url, name in zip(urls, names):
            image_path = f'./img/{name}.jpg'
            if not os.path.exists(image_path):
                futures.append(executor.submit(get_image_from_url, url, name))
                    
    for future in concurrent.futures.as_completed(futures):
        future.result()

download_all_images(product[["img",'شناسه']])

Invalid URL 'T': No scheme supplied. Perhaps you meant http://T? +++++ T
Invalid URL 'N': No scheme supplied. Perhaps you meant http://N? +++++ N
Invalid URL 'T': No scheme supplied. Perhaps you meant http://T? +++++ T
Invalid URL "'": No scheme supplied. Perhaps you meant http://'? +++++ '
Invalid URL 'T': No scheme supplied. Perhaps you meant http://T? +++++ T
Invalid URL 'N': No scheme supplied. Perhaps you meant http://N? +++++ N
